##### <font color='green'> TODO </font>:
- [x] make a generic function for Diagnostic category;
- [x] output chemical, protein, genetic biomarker types;
- how it can be linked to the organ (?);
- do it with bulks;
- find a tool to create a DB schema;
(https://online.visual-paradigm.com/pricing/)
- create DB schema;
- add Complete Blood Count;
- calculate how it's expensive (gigas);
- find a good database (Aurora?);
- populate DB;
- cron job to do it automatically.

## <font color='red'> BioMarkers </font>

category: {Diagnostic, Prognostic, Predictive, Exposure}

biomarker_type: {Chemical, Protein, Genetic, Karyotype}

In [2]:
import os
import requests

from pprint import pprint

In [3]:
! which python3

/Users/eolariu/miniconda3/envs/mybio/bin/python3


In [4]:
# api_key_val = os.environ.get("api_key")
api_key_val = os.environ['API_KEY']
# print(os.environ)

In [5]:
def output_all_conditions(url, category, biomarker_type, start_page, end_page):
    conditions = {}
    
    for page in range(start_page, end_page+1):
        names = []
        parameters = {'api_key': api_key_val,
                      'category': category,
                      'biomarker_type': biomarker_type,
                      'page': page}

        r = requests.get(url, params=parameters)

        response = r.json()

        for key, value in response['biomarkers'].items():
            for biomarker in value:
                biomarker_name, condition_name = biomarker['biomarker_name'], biomarker['condition_name']
                if condition_name in conditions:
                    conditions[condition_name] += [biomarker_name]
                else:
                    conditions[condition_name] = [biomarker_name]
    return conditions

## Diagnostic-Chemical

In [6]:
start_page = 1
end_page = 132
category = 'Diagnostic'
biomarker_type = 'Chemical'

url = 'http://markerdb.ca/api/v1/generalapi/generalrequest'

In [7]:
conditions_diagnostic_chemicals = output_all_conditions(url, 
                                                        category, 
                                                        biomarker_type, 
                                                        start_page, 
                                                        end_page)

In [8]:
for key, value in conditions_diagnostic_chemicals.items():
    print(key, value)
    print()

Normal ['1,3-Diaminopropane', '1-Methylhistidine', '1,3-Diaminopropane', '1-Methylhistidine', '1-Methylhistidine', '1,3-Diaminopropane']

Leukemia ['1,3-Diaminopropane', 'L-Glutamine', 'L-Histidine', 'L-Tryptophan', 'L-Arginine', 'L-Glutamic acid', 'Putrescine', 'Spermine', 'Spermidine', 'N1-Acetylspermine', 'N-Acetylputrescine', 'L-Asparagine', 'L-Asparagine', 'L-Histidine', '1,3-Diaminopropane', 'L-Glutamic acid', '1,3-Diaminopropane', 'L-Asparagine']

Alzheimer's Disease ['1-Methylhistidine', 'DHEA sulfate', 'L-Cystine', 'L-Valine', 'Citrulline', 'Hydroxyproline', 'L-Leucine', 'Glycine', 'L-Glutamine', 'L-Threonine', 'L-Lysine', 'Ornithine', 'Carnosine', 'Glycylproline', 'L-Histidine', 'L-Isoleucine', 'Anserine', 'L-Tryptophan', 'L-Cystathionine', 'L-Phenylalanine', 'Cytidine', 'L-Serine', 'Beta-Alanine', 'Guanine', 'L-Malic acid', 'Hypoxanthine', 'cis-Aconitic acid', 'Oxidized glutathione', 'Isocitric acid', 'D-Ribulose 5-phosphate', 'L-Alanine', "Uridine 5'-monophosphate", 'Glycer

In [9]:
len(conditions_diagnostic_chemicals.keys())

306

In [10]:
for k in conditions_diagnostic_chemicals.keys():
    print(k)

Normal
Leukemia
Alzheimer's Disease
Pregnancy
Chronic Kidney Disease
Kidney Disease
Obesity
Preeclampsia/Eclampsia
11 Beta Hydroxylase Deficiency
Congenital Adrenal Hyperplasia
Polycystic Ovary Syndrome
Rheumatoid Arthritis
Lipoid Congenital Adrenal Hyperplasia
Congenital Adrenal Insufficiency with 46,XY Sex Reversal
Cirrhosis
Diabetes Mellitus Type 2
X-Linked Ichthyosis
21 Hydroxylase Deficiency
Cadmium Exposure
Aromatase Deficiency
Acute Myelogenous Leukemia
Fanconi Syndrome
Fructose 1,6 Bisphosphatase Deficiency
Carnitine Palmitoyltransferase I Deficiency
Familial Hyperinsulinemic Hypoglycemia
Acute Myeloid Leukemia
3 Hydroxyacyl Co A Dehydrogenase Deficiency
Lipodystrophy
Type I Autosonal Dominant Pseudyhopoaldosteronism
Type 4A Bartter Syndrome
Type 4B Bartter Syndrome
Antley Bixler Syndrome With Genital Anomalies And Disordered Steroidogenesis
Canavan Disease
Chronic Renal Failure
Lesch Nyhan Syndrome
Uremia
Lewy Body Dementia
Brown Vialetto Van Laere Syndrome
Antenatal Bartter S

## Diagnostic-Protein

In [13]:
start_page = 1
end_page = 500
# 2557
category = 'Diagnostic'
biomarker_type = 'Protein'

url = 'http://markerdb.ca/api/v1/generalapi/generalrequest'

In [15]:
# conditions_diagnostic_proteins = output_all_conditions(url,
#                                                        category,
#                                                        biomarker_type,
#                                                        start_page,
#                                                        end_page)

In [ ]:
for key, value in conditions_diagnostic_proteins.items():
    print(key, value)
    print()

## Diagnostic-Genetic

In [22]:
start_page = 1
end_page = 2556
# 2556
category = 'Diagnostic'
biomarker_type = 'Genetic'

url = 'http://markerdb.ca/api/v1/generalapi/generalrequest'

In [ ]:
conditions_diagnostic_genetics = output_all_conditions(url,
                                                       category,
                                                       biomarker_type,
                                                       start_page,
                                                       end_page)

In [ ]:
for key, value in conditions_diagnostic_genetics.items():
    print(key, value)
    print()

## Protein biomarkers search by ID.

In [120]:
protein_biomarkers_dict = {}

base = 'MDB'
start_marker_number = 873
end_marker_number = 56533

# MDB00000873
# MDB00056533

# for number in range(start_marker_number-1, end_marker_number):
# number_str = base + str(number).zfill(8)
number_str = 'MDB00000873'

parameters = {'api_key': api_key_val,
              'markerdb_id': number_str}

r = requests.get('http://markerdb.ca/api/v1/proteinapi/proteinrequest', params=parameters)
try:
    response = r.json()['protein']
    name = response['name']
    general_function = response['general_function']
    protein_biomarkers_dict[name] = [number_str, general_function]
except KeyError:
    pass


In [121]:
protein_biomarkers_dict

{'Glycated hemoglobin': ['MDB00000873',
  'Involved in oxygen transport from the lung to the various peripheral tissues. [Uniprot]']}